# Load segments

In this we demonstrate loading of a sample data file [reduced_sample_data.json](../../data/reduced_sample_data.json) and showcase all the 
various resources that gets populated.

#### Pre-requisite
It is assumed that the setup steps are completed successfully. These are:
 - creating the database, schemas, stages
 - defining the external stage
 - defining the functions and stored procedures
 - defining the external tables and views

If you had not done this, use the streamlit (./bin/run_app.sh) to create these as defined in the subpage 'Setup'

In [11]:
# Initialization block
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,VSEKAR,PUBLIC,INDSOL_CMSGOV_PRICING,PUBLIC


In [12]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

display(Markdown("### Cleanup block"))
# We will cleanup specific resources and artifacts from possible previous runs.

stmts = [
    f''' truncate table segment_task_execution_status; '''
    ,f''' truncate table task_to_segmentids; '''
    ,f''' truncate table in_network_rates_file_header; '''
    ,f''' truncate table in_network_rates_segment_header; '''
    ,f''' alter stage {config['APP_DB']['ext_stage']} refresh; '''
]    
    
print(' truncating tables ...')
for stmt in stmts:
    sp_session.sql(stmt).collect()

print(f''' cleaning up files in external stage under path {config['APP_DB']['folder_parsed']}/reduced_sample_data/ ...''')

stmt = f''' select relative_path from directory(@{config['APP_DB']['ext_stage']}) where relative_path like '%{config['APP_DB']['folder_parsed']}/reduced_sample_data/%'; '''
files = sp_session.sql(stmt).collect()
for r in files:
    stmt = f''' remove @{config['APP_DB']['ext_stage']}/{r['RELATIVE_PATH']}; '''
    sp_session.sql(stmt).collect()

### Cleanup block

 truncating tables ...
 cleaning up files in external stage under path raw_parsed/reduced_sample_data/ ...


---
## Data loading
We will be loading the segments and also file header and show case how this functions without the dag.

In [13]:
df = sp_session.call('parse_negotiation_arrangement_segments' ,'data_stg/data','reduced_sample_data.json',f"@{config['APP_DB']['ext_stage']}/{config['APP_DB']['folder_parsed']}" ,0 ,100)

sp_session.sql(f''' alter stage {config['APP_DB']['ext_stage']} refresh; ''').collect()
print(' Status of execution')
print(df)

 Status of execution
{
  "EOF_Reached": true,
  "data_file": "reduced_sample_data.json",
  "elapsed": "=> 0:00:34.921314 ",
  "end_rec_num": 100,
  "last_seg_no": 10,
  "start_rec_num": 0,
  "status": true
}


In [15]:
print(' Parsing file headers')
df = sp_session.call('parse_file_header' ,'data_stg/data','reduced_sample_data.json')

print(' Status of execution')
print(df)

 Parsing file headers
 Status of execution
{
  "data_file": "reduced_sample_data.json",
  "elapsed": "=> 0:00:04.116450 ",
  "status": true
}


---
## Inspection

In [16]:
display(Markdown("The table in_network_rates_file_header holds the file header elements"))

df = (sp_session.table('in_network_rates_file_header')
        .sample(n=5)
        .to_pandas())

display(df)

The table in_network_rates_file_header holds the file header elements

,DATA_FILE,HEADER,INSERTED_AT
0,reduced_sample_data.json,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""last_updated_on"": ""2022-10-01"",\n ""reporting_entity_name"": ""Priority Health"",\n ""reporting_entity_type"": ""health insurance issuer"",\n ""version"": ""http://json-schema.org/draft-07/schema#""\n}",2022-12-30 15:35:05.661


In [17]:
print(' The HEADER has the following data ')
json.loads(df['HEADER'][0])

 The HEADER has the following data 


{'DATA_FILE': 'reduced_sample_data.json',
 'last_updated_on': '2022-10-01',
 'reporting_entity_name': 'Priority Health',
 'reporting_entity_type': 'health insurance issuer',
 'version': 'http://json-schema.org/draft-07/schema#'}

In [5]:
display(Markdown("The table in_network_rates_segment_header lists the various segments that were loaded"))

df = (sp_session.table('in_network_rates_segment_header')
        .sample(n=5)
        .to_pandas())

display(df)

The table in_network_rates_segment_header lists the various segments that were loaded

,DATA_FILE,SEGMENT_ID,NEGOTIATED_RATES_INFO,NEGOTIATED_RATES_COUNT,BUNDLED_CODES_COUNT,COVERED_SERVICES_COUNT,INSERTED_AT
0,reduced_sample_data.json,ffs::cpt::33235::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::33235::"",\n ""SEQ_NO"": 8,\n ""billing_code"": ""33235"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Removal of Perm Pacemaker; and Trnsvenous Electrodes, Dual Lead System"",\n ""name"": ""Removal of Perm Pacemaker; and Trnsvenous Electrodes, Dual Lead System"",\n ""negotiation_arrangement"": ""ffs""\n}",5029,-1,-1,2022-12-30 12:31:15.131
1,reduced_sample_data.json,ffs::cpt::90838::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::90838::"",\n ""SEQ_NO"": 2,\n ""billing_code"": ""90838"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Psychotherapy, 60 minutes with patient when performed with an evaluation and management service (List separately in addition to the code for primary procedure)"",\n ""name"": ""Psychotherapy, 60 minutes with patient when performed with an evaluation and management service (List separately in addition to the code for primary procedure)"",\n ""negotiation_arrangement"": ""ffs""\n}",4843,-1,-1,2022-12-30 12:30:56.976
2,reduced_sample_data.json,ffs::cpt::76391::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::76391::"",\n ""SEQ_NO"": 5,\n ""billing_code"": ""76391"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Magnetic resonance (eg, vibration) elastography"",\n ""name"": ""Magnetic resonance (eg, vibration) elastography"",\n ""negotiation_arrangement"": ""ffs""\n}",1784,-1,-1,2022-12-30 12:31:05.931
3,reduced_sample_data.json,ffs::cpt::a4614::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::a4614::"",\n ""SEQ_NO"": 3,\n ""billing_code"": ""A4614"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Peak expiratory flow rate meter hand held"",\n ""name"": ""Peak expiratory flow rate meter hand held"",\n ""negotiation_arrangement"": ""ffs""\n}",235,-1,-1,2022-12-30 12:31:00.028
4,reduced_sample_data.json,ffs::cpt::62322::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::62322::"",\n ""SEQ_NO"": 1,\n ""billing_code"": ""62322"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Injection(s), of diagnostic or therapeutic substance(s) (eg, anesthetic, antispasmodic, opioid, steroid, other solution), not including neurolytic substances, including needle"",\n ""name"": ""Injection(s), of diagnostic or therapeutic substance(s) (eg, anesthetic, antispasmodic, opioid, steroid, other solution), not including neurolytic substances, including needle"",\n ""negotiation_arrangement"": ""ffs""\n}",4170,-1,-1,2022-12-30 12:30:54.047


In [6]:
print(' The NEGOTIATED_RATES_INFO has the following data ')
json.loads(df['NEGOTIATED_RATES_INFO'][0])

 The NEGOTIATED_RATES_INFO has the following data 


{'DATA_FILE': 'reduced_sample_data.json',
 'SEGMENT_ID': 'ffs::cpt::33235::',
 'SEQ_NO': 8,
 'billing_code': '33235',
 'billing_code_type': 'CPT',
 'billing_code_type_version': '',
 'description': 'Removal of Perm Pacemaker; and Trnsvenous Electrodes, Dual Lead System',
 'name': 'Removal of Perm Pacemaker; and Trnsvenous Electrodes, Dual Lead System',
 'negotiation_arrangement': 'ffs'}

In [7]:
para = f''' 
The parsed data are stored as parquet files in the external stage at: @{config['APP_DB']['ext_stage']}/{config['APP_DB']['folder_parsed']}. The directory structure follows the format:

<ext stage>/<folder_parsed>/<data file basename (ex: reduced_sample_data)/<segment_id>/<segment type (negotiated_rates)>/data_<seq_no>_<chunk_no>.parquet.gz

Description:
- ext stage : external stage name
- folder_parsed : configured value of APP_DB.folder_parsed in config.ini
- data file basename : the data file basename, without the file extension
- segment_id : a unique identifier for the segment, this is a composite key of <negotiation_arrangement>::<billing_code_type>::<billing_code>::<billing_code_type_version>
- segment type : indicates the segment children type, this could be either of negotiated_rates/bundled_codes/covered_services
- The file which will contain the record, stored in parquet file.
'''
print(para)

stmt = f''' select relative_path from directory(@{config['APP_DB']['ext_stage']}) where relative_path like '%{config['APP_DB']['folder_parsed']}/reduced_sample_data%' limit 5; '''
df = sp_session.sql(stmt).to_pandas()
display(df)

 
The parsed data are stored as parquet files in the external stage at: @ext_data_stg/raw_parsed. The directory structure follows the format:

<ext stage>/<folder_parsed>/<data file basename (ex: reduced_sample_data)/<segment_id>/<segment type (negotiated_rates)>/data_<seq_no>_<chunk_no>.parquet.gz

Description:
- ext stage : external stage name
- folder_parsed : configured value of APP_DB.folder_parsed in config.ini
- data file basename : the data file basename, without the file extension
- segment_id : a unique identifier for the segment, this is a composite key of <negotiation_arrangement>::<billing_code_type>::<billing_code>::<billing_code_type_version>
- segment type : indicates the segment children type, this could be either of negotiated_rates/bundled_codes/covered_services
- The file which will contain the record, stored in parquet file.



,RELATIVE_PATH
0,raw_parsed/reduced_sample_data/ffs::cpt::15004::/negotiated_rates/data_7_0.parquet.gz
1,raw_parsed/reduced_sample_data/ffs::cpt::15004::/negotiated_rates/data_7_1.parquet.gz
2,raw_parsed/reduced_sample_data/ffs::cpt::33235::/negotiated_rates/data_8_0.parquet.gz
3,raw_parsed/reduced_sample_data/ffs::cpt::33235::/negotiated_rates/data_8_1.parquet.gz
4,raw_parsed/reduced_sample_data/ffs::cpt::62322::/negotiated_rates/data_1_0.parquet.gz


In [8]:
para = f''' 
The parquet file can be queried inside Snowflake via external table: ext_negotiated_arrangments_staged. Each of the folder structure is partitioned to seperate columns , which can help with pruning to specific segments.
The value contains the negotiated_rates and other children elements stored in JSON format
'''
print(para)

pd.set_option('display.max_colwidth', 25)
df = (sp_session.table('ext_negotiated_arrangments_staged')
        .sample(n=5)
        .to_pandas())

display(df)


 
The parquet file can be queried inside Snowflake via external table: ext_negotiated_arrangments_staged. Each of the folder structure is partitioned to seperate columns , which can help with pruning to specific segments.
The value contains the negotiated_rates and other children elements stored in JSON format



,VALUE,P_DATA_FL,P_SEGMENT_ID,P_NEGOTIATION_ARRANGEMENT,P_BILLING_CODE_TYPE,P_BILLING_CODE,P_BILLING_CODE_TYPE_VERSION,P_SEGMENT_TYPE
0,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::85041::,ffs,cpt,85041,,negotiated_rates
1,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::62322::,ffs,cpt,62322,,negotiated_rates
2,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::76391::,ffs,cpt,76391,,negotiated_rates
3,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::33235::,ffs,cpt,33235,,negotiated_rates
4,"{\n ""CHUNK_NO"": 1,\n...",reduced_sample_data,ffs::cpt::15004::,ffs,cpt,15004,,negotiated_rates


In [9]:
print(' a sample view of one of the records')
j = json.loads(df['VALUE'][0])
print(j)

 a sample view of one of the records
{'CHUNK_NO': 0, 'DATA_FILE': 'reduced_sample_data.json', 'NEGOTIATED_RATES': [{'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 1.51, 'negotiated_type': 'negotiated', 'service_code': ['11']}], 'provider_groups': [{'npi': [1902809940], 'tin': {'type': 'ein', 'value': '133757370'}}]}, {'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 1.51, 'negotiated_type': 'negotiated', 'service_code': ['11']}], 'provider_groups': [{'npi': [1104829159], 'tin': {'type': 'ein', 'value': '840611484'}}]}, {'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 2.65, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}], 'provider_groups': [{'npi': [1245307818], 'tin': {'type': 'ein', 'value': '520890739'}}]}, {'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-

--- 
### Closeout

    With that we are finished this section of the demo setup

In [10]:
sp_session.close()
print('Finished!!!')

Finished!!!
